In [23]:
from fosforml.model_manager.snowflakesession import get_session

In [24]:
my_session = get_session()

In [25]:
table_name = 'Fact_Customer_Orders'
sf_df = my_session.sql("select * from {}".format(table_name)) 
 
df = sf_df.to_pandas() 
print(df.shape)

(225563, 4)


In [26]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [27]:
my_session.connection.database 

'MFG_DEMAND_ANALYTICS'

In [28]:
my_session.connection.schema

'GOLD'

In [29]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [30]:
type(df)

pandas.core.frame.DataFrame

In [31]:
df.head(5)

,ORDER_ID,PRODUCT_ID,ORDER_QTY,ORDER_DATE
0,Order_000006,Product_001,850,2019-01-02
1,Order_000001,Product_004,700,2019-01-02
2,Order_000003,Product_004,900,2019-01-02
3,Order_000008,Product_004,550,2019-01-02
4,Order_000012,Product_004,300,2019-01-02


In [32]:
df.describe()

,ORDER_QTY
count,225563.000000
mean,525.099418
std,288.681633
min,50.000000
25%,250.000000
50%,550.000000
75%,800.000000
max,1000.000000


In [33]:
df.isnull().sum()

ORDER_ID      0
PRODUCT_ID    0
ORDER_QTY     0
ORDER_DATE    0
dtype: int64

In [34]:
df.duplicated().sum()

0

In [35]:
df.corr(numeric_only=True)

,ORDER_QTY
ORDER_QTY,1.0


In [36]:
import matplotlib.pyplot as plt 

In [37]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [38]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 


In [39]:
table_name = 'Fact_Customer_Orders'
sf_df = my_session.sql("select * from {}".format(table_name)) 
 
df = sf_df.to_pandas() 

In [40]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [41]:
data = df[['PRODUCT_ID','ORDER_QTY', 'ORDER_DATE']]

In [42]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_QTY   225563 non-null  int16         
 2   ORDER_DATE  225563 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [44]:
data = data.set_index('ORDER_DATE')

In [45]:
data.sort_index(inplace=True)

data

In [46]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [47]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'test_forecast_order_qty','rse', 'rmse','r2_score'])

In [48]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'forecast_order_qty'])

In [ ]:
for PRODUCT_ID, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    
    date_list = test_df.index.to_list()
    
    new_row = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list,
        'test_forecast_order_qty': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.9/site-packag

In [ ]:
test_results

In [ ]:
forecast_results